In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,classification_report

In [14]:
df = pd.read_csv(r'Car_Sales_Prediction.csv')
df.head()

,Name,Age,Gender,Salary_M,Dep_Family_Members,Pref_Fuel_Type,Sold
0,Mohan,24,Male,70000,0,Diesel,0.0
1,Saurabh,43,Male,55000,1,Petrol,1.0
2,Nilesh,36,Male,45000,2,Petrol,0.0
3,Viart,30,Male,66000,0,Petrol,1.0
4,Sachin,42,Male,71000,2,Diesel,1.0


In [15]:
df['Sold'].value_counts()

1.0    8
0.0    7
Name: Sold, dtype: int64

In [16]:
df['Salary_M'].value_counts()

70000    1
55000    1
49000    1
98000    1
83000    1
77000    1
62000    1
39000    1
52000    1
47000    1
41000    1
25000    1
88000    1
91000    1
46000    1
23000    1
90000    1
80000    1
71000    1
66000    1
45000    1
59000    1
Name: Salary_M, dtype: int64

In [20]:
df_model=df[['Age','Gender','Dep_Family_Members','Pref_Fuel_Type','Sold']]
df_model

,Age,Gender,Dep_Family_Members,Pref_Fuel_Type,Sold
0,24,Male,0,Diesel,0.0
1,43,Male,1,Petrol,1.0
2,36,Male,2,Petrol,0.0
3,30,Male,0,Petrol,1.0
4,42,Male,2,Diesel,1.0
5,55,Female,2,Diesel,0.0
6,38,Female,2,Diesel,1.0
7,45,Male,4,Petrol,0.0
8,37,Female,3,Diesel,1.0
9,55,Female,2,Petrol,0.0


In [21]:
data_gender =pd.get_dummies(df_model['Gender'],drop_first=True)
data_gender.rename(columns={'Male':'Gender_Male'},inplace=True)
data_gender.head(6)

,Gender_Male
0,1
1,1
2,1
3,1
4,1
5,0


In [22]:
data_fuel = pd.get_dummies(df_model['Pref_Fuel_Type'],drop_first=True)
data_fuel.rename(columns={'Petrol':'Fuel_Petrol'},inplace=True)
data_fuel.head(6)

,Fuel_Petrol
0,0
1,1
2,1
3,1
4,0
5,0


In [24]:
data_fam = pd.get_dummies(df_model['Dep_Family_Members'],drop_first=True)
data_fam.rename(columns={1:'fam_1',2:'fam_2',3:'fam_3',4:'fam_4',5:'fam_5'},inplace=True)

In [25]:
data_fam.head()

,fam_1,fam_2,fam_3,fam_4,fam_5
0,0,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,0,0,0
4,0,1,0,0,0


In [36]:
data =pd.concat([df_model ,data_gender,data_fuel,data_fam],axis=1)
data.head()

,Age,Gender,Dep_Family_Members,Pref_Fuel_Type,Sold,Gender_Male,Fuel_Petrol,fam_1,fam_2,fam_3,fam_4,fam_5
0,24,Male,0,Diesel,0.0,1,0,0,0,0,0,0
1,43,Male,1,Petrol,1.0,1,1,1,0,0,0,0
2,36,Male,2,Petrol,0.0,1,1,0,1,0,0,0
3,30,Male,0,Petrol,1.0,1,1,0,0,0,0,0
4,42,Male,2,Diesel,1.0,1,0,0,1,0,0,0


In [37]:
data.drop(['Gender','Dep_Family_Members','Pref_Fuel_Type'],axis=1,inplace=True)
data

,Age,Sold,Gender_Male,Fuel_Petrol,fam_1,fam_2,fam_3,fam_4,fam_5
0,24,0.0,1,0,0,0,0,0,0
1,43,1.0,1,1,1,0,0,0,0
2,36,0.0,1,1,0,1,0,0,0
3,30,1.0,1,1,0,0,0,0,0
4,42,1.0,1,0,0,1,0,0,0
5,55,0.0,0,0,0,1,0,0,0
6,38,1.0,0,0,0,1,0,0,0
7,45,0.0,1,1,0,0,0,1,0
8,37,1.0,0,0,0,0,1,0,0
9,55,0.0,0,1,0,1,0,0,0


In [38]:
#split known and unknown
known_data  = data[pd.notnull(data['Sold'])]
print(known_data)

    Age  Sold  Gender_Male  Fuel_Petrol  fam_1  fam_2  fam_3  fam_4  fam_5
0    24   0.0            1            0      0      0      0      0      0
1    43   1.0            1            1      1      0      0      0      0
2    36   0.0            1            1      0      1      0      0      0
3    30   1.0            1            1      0      0      0      0      0
4    42   1.0            1            0      0      1      0      0      0
5    55   0.0            0            0      0      1      0      0      0
6    38   1.0            0            0      0      1      0      0      0
7    45   0.0            1            1      0      0      0      1      0
8    37   1.0            0            0      0      0      1      0      0
9    55   0.0            0            1      0      1      0      0      0
10   29   0.0            1            0      0      0      0      0      1
11   44   1.0            1            0      1      0      0      0      0
12   33   1.0            

In [39]:
unknown_data =data[pd.isnull(data['Sold'])]

In [40]:
print(unknown_data)

    Age  Sold  Gender_Male  Fuel_Petrol  fam_1  fam_2  fam_3  fam_4  fam_5
15   27   NaN            0            0      0      0      1      0      0
16   32   NaN            0            1      0      1      0      0      0
17   35   NaN            1            0      0      0      0      0      0
18   38   NaN            1            1      0      0      0      1      0
19   41   NaN            1            0      0      1      0      0      0
20   30   NaN            1            1      1      0      0      0      0
21   43   NaN            1            0      1      0      0      0      0


In [41]:
known_data_x=known_data.drop(['Sold'],axis=1)
known_data_y=known_data['Sold']

In [42]:
#build the model

In [56]:
log_reg=LogisticRegression()

In [57]:
log_reg.fit(known_data_x,known_data_y)

LogisticRegression()

In [58]:
predict_known_Y=log_reg.predict(known_data_x)

In [59]:
for x,y,pred_y in zip(known_data_x.values,known_data_y,predict_known_Y):
    print(x,y,pred_y,sep='--')

[24  1  0  0  0  0  0  0]--0.0--1.0
[43  1  1  1  0  0  0  0]--1.0--1.0
[36  1  1  0  1  0  0  0]--0.0--0.0
[30  1  1  0  0  0  0  0]--1.0--1.0
[42  1  0  0  1  0  0  0]--1.0--0.0
[55  0  0  0  1  0  0  0]--0.0--0.0
[38  0  0  0  1  0  0  0]--1.0--1.0
[45  1  1  0  0  0  1  0]--0.0--0.0
[37  0  0  0  0  1  0  0]--1.0--1.0
[55  0  1  0  1  0  0  0]--0.0--1.0
[29  1  0  0  0  0  0  1]--0.0--0.0
[44  1  0  1  0  0  0  0]--1.0--1.0
[33  0  1  0  1  0  0  0]--1.0--1.0
[23  1  0  0  0  1  0  0]--0.0--1.0
[39  0  1  1  0  0  0  0]--1.0--1.0


In [60]:
print(known_data_y,predict_known_Y,sep='=====')

0     0.0
1     1.0
2     0.0
3     1.0
4     1.0
5     0.0
6     1.0
7     0.0
8     1.0
9     0.0
10    0.0
11    1.0
12    1.0
13    0.0
14    1.0
Name: Sold, dtype: float64=====[1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1.]


In [62]:
#Display confusion matrix
print(confusion_matrix(known_data_y,predict_known_Y))
print(confusion_matrix(known_data_y,predict_known_Y).ravel())

[[4 3]
 [1 7]]
[4 3 1 7]


In [69]:
tn,fp,fn,tp=confusion_matrix(known_data_y,predict_known_Y).ravel()

In [70]:
print(tn,tp,fp,fn,sep=',')

4,7,3,1


In [71]:
#Accuracy Score

#sum of diagonalelements/sum of everything
accuracy =(tn+tp)/(tn+fp+fn+tp)

In [72]:
print('Accuracy',accuracy)

Accuracy 0.7333333333333333


In [74]:
#precision =>what propertion of +ve identification was actually identified
#Tp/(Tp+Fp)  ,,,,,  Tn/(Tn+Fn)

In [75]:
print('precision_score',precision_score(known_data_y,predict_known_Y))

precision_score 0.7


In [76]:
#Recall what proportions of actual positivies identifies correctly
# Tp/(Tp+Fn) , Tn/(Tn+Fp)

In [77]:
print('Recall-',recall_score(known_data_y,predict_known_Y))

Recall- 0.875


In [78]:
print(classification_report(known_data_y,predict_known_Y))

              precision    recall  f1-score   support

         0.0       0.80      0.57      0.67         7
         1.0       0.70      0.88      0.78         8

    accuracy                           0.73        15
   macro avg       0.75      0.72      0.72        15
weighted avg       0.75      0.73      0.73        15

